# CCTV Video Processing - Google Colab

This notebook processes CCTV videos to detect and recognize people using YOLO and InsightFace.

## Setup Instructions:
1. **Upload your videos folder** - Run the upload cell below
2. **Upload new_photos folder** (optional) - To build face recognition database
   - Structure: `new_photos/person_name/image1.jpg, image2.jpg, ...`
   - Example: `new_photos/ayush/ayush-1.jpg`
3. **Or upload face_db folder** (alternative) - If you already have a face database
4. **Run all cells** - Execute cells sequentially

## What this notebook does:
- Creates face recognition database from photos (if new_photos provided)
- Detects people in videos using YOLO
- Recognizes faces using InsightFace
- Tracks people across frames
- Generates annotated videos and detection JSON files


In [ ]:
# Install dependencies
%pip install -q opencv-python torch torchvision insightface deep-sort-realtime numpy pillow lancedb pyarrow ultralytics onnxruntime onnxruntime-gpu

print("✓ Dependencies installed!")


## Step 1: Upload Videos Folder

Upload your videos folder. The folder should contain video files (e.g., `.mp4`, `.avi`, etc.)


In [ ]:
from google.colab import files
import zipfile
import os
from pathlib import Path

# Create directories
os.makedirs("cctv_videos", exist_ok=True)
os.makedirs("face_db", exist_ok=True)
os.makedirs("processed_data/videos", exist_ok=True)
os.makedirs("processed_data/individual", exist_ok=True)

print("📁 Directories created!")
print("\n📤 Now upload your videos folder:")
print("   Option 1: Upload a zip file containing your videos")
print("   Option 2: Upload individual video files")
print("\n💡 Tip: You can also upload face_db folder if you have one")


## Step 1b: Upload Face Photos (Optional)

Upload your `new_photos` folder to build the face recognition database.  
**Structure**: `new_photos/person_name/image1.jpg, image2.jpg, ...`

Example:
- `new_photos/ayush/ayush-1.jpg`
- `new_photos/kanika/kanika-1.jpg`


In [ ]:
# Upload new_photos folder (zip file)
os.makedirs("new_photos", exist_ok=True)

print("📤 Upload new_photos folder:")
print("   Option 1: Upload a zip file containing new_photos folder")
print("   Option 2: Skip this step if you already have face_db folder")
print()

uploaded_photos = files.upload()

# Extract new_photos if uploaded
for filename in uploaded_photos.keys():
    if filename.endswith('.zip'):
        print(f"📦 Extracting {filename}...")
        with zipfile.ZipFile(filename, 'r') as zip_ref:
            # Check if it contains new_photos folder
            if any('new_photos' in f for f in zip_ref.namelist()):
                zip_ref.extractall(".")
                print(f"✓ Extracted new_photos folder")
            else:
                # Extract to new_photos directory
                zip_ref.extractall("new_photos")
                print(f"✓ Extracted to new_photos/")

# List uploaded photos
new_photos_path = Path("new_photos")
if new_photos_path.exists():
    person_dirs = [d for d in new_photos_path.iterdir() if d.is_dir()]
    if person_dirs:
        print(f"\n📸 Found {len(person_dirs)} person folder(s):")
        for person_dir in sorted(person_dirs):
            image_files = list(person_dir.glob("*.jpg")) + list(person_dir.glob("*.jpeg")) + list(person_dir.glob("*.png"))
            print(f"   - {person_dir.name}/ ({len(image_files)} images)")
    else:
        print("\n⚠ No person folders found in new_photos/")
        print("   Expected structure: new_photos/person_name/image1.jpg")
else:
    print("\n⚠ new_photos folder not found")
    print("   You can skip face recognition or upload face_db folder instead")


## Step 1c: Create Face Database from Photos

This will process the uploaded photos and create/update the face database.


In [ ]:
import cv2
import numpy as np
import lancedb
from pathlib import Path
from insightface.app import FaceAnalysis

def process_image_for_database(image_path, person_name, app):
    """Process a single image and return embedding with label"""
    img = cv2.imread(str(image_path))
    if img is None:
        return None
    
    # Detect faces using InsightFace
    faces = app.get(img)
    if len(faces) == 0:
        return None
    
    # Use first face found
    face = faces[0]
    embedding = face.embedding  # 512-dimensional embedding
    
    return {
        "label": person_name,
        "embedding": np.asarray(embedding, dtype=np.float32).flatten()
    }

def populate_face_database(new_photos_dir="new_photos"):
    """Process all images in new_photos and add to database"""
    new_photos_path = Path(new_photos_dir)
    
    if not new_photos_path.exists():
        print(f"⚠ {new_photos_dir} folder not found. Skipping face database creation.")
        print("   Face recognition will return 'unknown' for all faces.")
        return False
    
    print("\n" + "="*60)
    print("PROCESSING FACE PHOTOS")
    print("="*60)
    
    # Initialize InsightFace
    print("Loading InsightFace model...")
    app = FaceAnalysis(providers=["CUDAExecutionProvider", "CPUExecutionProvider"], 
                      allowed_modules=["detection", "recognition"])
    app.prepare(ctx_id=0, det_size=(640, 640))
    print("✓ InsightFace loaded")
    
    # Create or get face table
    db = lancedb.connect("face_db")
    face_table = None
    table_needs_creation = True
    
    try:
        face_table = db.open_table("face_data")
        print("✓ Opened existing face database")
        table_needs_creation = False
    except:
        print("📝 Will create new face database")
        table_needs_creation = True
    
    total_processed = 0
    total_added = 0
    total_failed = 0
    
    # Process each person's directory
    person_dirs = [d for d in new_photos_path.iterdir() if d.is_dir()]
    
    if not person_dirs:
        print(f"⚠ No person folders found in {new_photos_dir}/")
        return False
    
    all_embeddings = []  # Collect all embeddings first
    
    for person_dir in sorted(person_dirs):
        person_name = person_dir.name
        print(f"\n👤 Processing: {person_name}/")
        
        # Get all image files
        image_files = []
        for ext in ['*.jpg', '*.jpeg', '*.png', '*.JPG', '*.JPEG', '*.PNG']:
            image_files.extend(person_dir.glob(ext))
        
        if len(image_files) == 0:
            print(f"  ⚠ No images found")
            continue
        
        for image_path in sorted(image_files):
            total_processed += 1
            print(f"  Processing: {image_path.name}...", end=" ")
            
            result = process_image_for_database(image_path, person_name, app)
            
            if result is None:
                print("✗ (no face detected)")
                total_failed += 1
                continue
            
            all_embeddings.append(result)
            print("✓")
    
    # Create table with first batch or add to existing
    if all_embeddings:
        try:
            if table_needs_creation:
                # Create table with first batch
                face_table = db.create_table("face_data", all_embeddings, mode="overwrite")
                print(f"\n✓ Created face database with {len(all_embeddings)} embeddings")
            else:
                # Add to existing table
                face_table.add(all_embeddings)
                print(f"\n✓ Added {len(all_embeddings)} embeddings to database")
            
            total_added = len(all_embeddings)
        except Exception as e:
            print(f"\n✗ Error: {e}")
            import traceback
            traceback.print_exc()
            total_failed = len(all_embeddings)
    
    print("\n" + "="*60)
    print("SUMMARY")
    print("="*60)
    print(f"  Total images processed: {total_processed}")
    print(f"  Successfully added: {total_added}")
    print(f"  Failed: {total_failed}")
    
    # Show database stats
    try:
        df = face_table.to_pandas()
        print(f"\n  Database size: {len(df)} embeddings")
        print(f"  Unique people: {df['label'].nunique()}")
        print(f"  People: {', '.join(sorted(df['label'].unique()))}")
    except Exception as e:
        print(f"\n  Could not query database: {e}")
    
    print("\n✓ Face database ready!")
    return True

# Process photos if new_photos folder exists
if Path("new_photos").exists():
    populate_face_database("new_photos")
else:
    print("⚠ Skipping face database creation (new_photos folder not found)")
    print("   You can upload face_db folder instead, or process videos without face recognition")


## Step 2: Create Helper Module

This creates the helper.py file needed for face recognition.


In [ ]:
# Upload videos (zip file or individual files)
# If you upload a zip file, it will be extracted automatically
uploaded = files.upload()

# Extract zip files if any
for filename in uploaded.keys():
    if filename.endswith('.zip'):
        print(f"📦 Extracting {filename}...")
        with zipfile.ZipFile(filename, 'r') as zip_ref:
            # Extract to cctv_videos if it's a videos folder
            if 'video' in filename.lower() or any(f.endswith(('.mp4', '.avi', '.mov')) for f in zip_ref.namelist()):
                zip_ref.extractall("cctv_videos")
                print(f"✓ Extracted videos to cctv_videos/")
            # Extract face_db if it's a face database
            elif 'face' in filename.lower() or 'db' in filename.lower():
                zip_ref.extractall("face_db")
                print(f"✓ Extracted face database to face_db/")
            else:
                zip_ref.extractall(".")
                print(f"✓ Extracted {filename} to current directory")
    elif filename.endswith(('.mp4', '.avi', '.mov', '.mkv')):
        # Move video files to cctv_videos
        os.rename(filename, f"cctv_videos/{filename}")
        print(f"✓ Moved {filename} to cctv_videos/")

# List uploaded videos
video_files = list(Path("cctv_videos").glob("*.*"))
video_extensions = ['.mp4', '.avi', '.mov', '.mkv', '.flv', '.wmv']
video_files = [f for f in video_files if f.suffix.lower() in video_extensions]

print(f"\n📹 Found {len(video_files)} video file(s):")
for vf in video_files:
    size_mb = vf.stat().st_size / (1024 * 1024)
    print(f"   - {vf.name} ({size_mb:.1f} MB)")


## Step 2: Create Helper Module

This creates the helper.py file needed for face recognition.


In [ ]:
# Create helper.py
helper_code = '''import numpy as np
import torch
import cv2  
from torchvision import transforms
from insightface.model_zoo import get_model
from insightface.utils import face_align
import torch.nn.functional as F
import lancedb
import os

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Initialize LanceDB connection
try:
    db = lancedb.connect("face_db")
    face_table = db.open_table("face_data")
    print("✓ Face database loaded")
except Exception as e:
    print(f"⚠ Warning: Could not load face database: {e}")
    print("   Face recognition will return 'unknown' for all faces")
    face_table = None

def vector_search(embedding, threshold):
    if embedding is None or face_table is None:
        return ("unknown", float("inf"))

    emb = np.asarray(embedding, dtype=np.float32)
    if emb.ndim != 1 or emb.shape[0] != 512:
        return ("unknown", float("inf"))

    try:
        res = (
            face_table
            .search(emb, vector_column_name="embedding")
            .metric("cosine")
            .select(["label", "_distance"])
            .limit(1)
            .to_list()
        )
    except Exception as e:
        return ("unknown", float("inf"))

    if not res:
        return ("unknown", float("inf"))

    label = res[0]["label"]
    dist = res[0]["_distance"]

    if label is None or dist is None or dist >= threshold:
        return ("unknown", dist)

    return (label, dist)
'''

with open("helper.py", "w") as f:
    f.write(helper_code)

print("✓ helper.py created!")


In [ ]:
# Camera configuration
# Update this to match your video files
# Format: "CAM_ID": {"name": "video_filename.mp4", "location": {"x": 0.0, "y": 1.0, "z": 0.0}}

CAMERA_CONFIG = {
    "CAM_01": {"name": "cp_lab1.mp4", "location": {"x": 0.0, "y": 1.0, "z": 0.0}},
    "CAM_02": {"name": "cp_lab2.mp4", "location": {"x": 0.866, "y": 0.5, "z": 0.0}},
    "CAM_03": {"name": "vlsi.mp4", "location": {"x": 0.866, "y": -0.5, "z": 0.0}},
    "CAM_04": {"name": "iot.mp4", "location": {"x": 0.0, "y": -1.0, "z": 0.0}},
    "CAM_05": {"name": "lift.mp4", "location": {"x": -0.866, "y": -0.5, "z": 0.0}},
    "CAM_06": {"name": "loby.mp4", "location": {"x": -0.866, "y": 0.5, "z": 0.0}},
}

# Auto-detect video files and create config
from pathlib import Path
video_dir = Path("cctv_videos")
video_files = list(video_dir.glob("*.mp4")) + list(video_dir.glob("*.avi")) + list(video_dir.glob("*.mov"))

if video_files:
    print("📹 Auto-detected video files:")
    auto_config = {}
    for i, vf in enumerate(sorted(video_files), 1):
        cam_id = f"CAM_{i:02d}"
        auto_config[cam_id] = {
            "name": vf.name,
            "location": {"x": 0.0, "y": 1.0, "z": 0.0}  # Default location
        }
        print(f"   {cam_id}: {vf.name}")
    
    # Update CAMERA_CONFIG with auto-detected files
    CAMERA_CONFIG = auto_config
    print(f"\n✓ Using auto-detected configuration with {len(CAMERA_CONFIG)} cameras")
else:
    print("⚠ No video files found. Using default CAMERA_CONFIG.")
    print("   Make sure your videos are in cctv_videos/ folder")


## Step 4: Initialize Models

This will download and load YOLO and InsightFace models.


In [ ]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

import cv2
import torch
import json
import os
import gc
import time
from pathlib import Path
from collections import defaultdict
from ultralytics import YOLO
from insightface.app import FaceAnalysis
from deep_sort_realtime.deepsort_tracker import DeepSort
from helper import vector_search

# GPU setup
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f"🖥️  Using device: {device}")

# Initialize models
print("\n📥 Loading models (this may take a few minutes on first run)...")
app = FaceAnalysis(providers=["CUDAExecutionProvider", "CPUExecutionProvider"], 
                  allowed_modules=["detection", "recognition"])
app.prepare(ctx_id=0, det_size=(640, 640))
print("✓ InsightFace loaded")

yolo_model = YOLO("yolov8m.pt").to(device)
print("✓ YOLO loaded")

print("\n✅ All models ready!")


## Step 5: Configure Processing Settings

Adjust these settings based on your needs:
- `FRAME_SKIP`: Process every Nth frame (higher = faster but less accurate)
- `MAX_FRAMES`: Limit processing to first N frames (None = process all)


In [ ]:
# Processing settings
FRAME_SKIP = 2              # Process every Nth frame (2 = process every 2nd frame)
ENABLE_GPU_CLEANUP = True   # Clear GPU cache periodically
CLEANUP_INTERVAL = 50       # Clear GPU cache every N frames
MAX_FRAMES = None           # Limit total frames (None = process all, set to number to limit)
PAUSE_EVERY_N_FRAMES = 100  # Small pause every N frames (0 = no pause)
PAUSE_DURATION = 0.1        # Pause duration in seconds

# Directories
VIDEO_DIR = "cctv_videos"
OUTPUT_VIDEO_DIR = "processed_data/videos"
OUTPUT_JSON = "processed_data/detections.json"
INDIVIDUAL_JSON_DIR = "processed_data/individual"

# Ensure output directories exist
os.makedirs(OUTPUT_VIDEO_DIR, exist_ok=True)
os.makedirs(os.path.dirname(OUTPUT_JSON), exist_ok=True)
os.makedirs(INDIVIDUAL_JSON_DIR, exist_ok=True)

print("⚙️  Processing settings:")
print(f"   Frame skip: {FRAME_SKIP}")
print(f"   Max frames: {MAX_FRAMES if MAX_FRAMES else 'All'}")
print(f"   GPU cleanup: {'Enabled' if ENABLE_GPU_CLEANUP else 'Disabled'}")


## Step 6: Process Videos

This will process all videos and generate annotated videos + detection JSON files.


In [ ]:
def process_video(cam_id, video_path, camera_config):
    """Process a single video"""
    print(f"\n{'='*60}")
    print(f"Processing {cam_id}: {video_path.name}")
    print(f"{'='*60}")
    
    # Open video
    cap = cv2.VideoCapture(str(video_path))
    if not cap.isOpened():
        raise ValueError(f"Error: Could not open video file {video_path}")
    
    # Get video properties
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    print(f"Resolution: {width}x{height}, FPS: {fps:.2f}, Frames: {total_frames}")
    
    # Initialize tracker
    tracker = DeepSort(max_age=40, max_cosine_distance=0.6, max_iou_distance=0.8)
    
    # Prediction dict
    prediction_dict = defaultdict(lambda: {
        "name": "unknown",
        "predictions": []
    })
    
    # Output video writer
    output_video_path = os.path.join(OUTPUT_VIDEO_DIR, f"{cam_id}.mp4")
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))
    
    # Store all detections
    all_detections = []
    
    frame_id = 0
    processed_frame_count = 0
    
    print(f"\nProcessing...")
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Frame skipping
        if frame_id % FRAME_SKIP != 0:
            frame_id += 1
            out.write(frame)
            continue
        
        # Limit total frames
        if MAX_FRAMES and processed_frame_count >= MAX_FRAMES:
            print(f"\n⚠ Reached MAX_FRAMES limit ({MAX_FRAMES})")
            while cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    break
                out.write(frame)
            break
        
        clean_frame = frame.copy()
        timestamp = frame_id / fps if fps > 0 else 0.0
        
        # YOLO detection
        persons = yolo_model(frame)[0]
        
        identities = []
        for person in persons.boxes:
            if int(person.cls[0]) == 0:  # Class 0 = person
                x1, y1, x2, y2 = map(int, person.xyxy[0])
                conf = person.conf[0].cpu().numpy()
                identities.append(([x1, y1, x2 - x1, y2 - y1], conf))
        
        # Update tracker
        tracks = tracker.update_tracks(identities, frame=frame)
        
        for track in tracks:
            if not track.is_confirmed():
                continue
            
            track_id = track.track_id
            name = prediction_dict[track_id]["name"]
            score = track.get_det_conf()
            
            # Get bounding box
            t, l, b, r = map(int, track.to_tlbr())
            
            # Face recognition for unknown persons
            if name == "unknown":
                person_crop = clean_frame[t:b, l:r]
                if person_crop.size != 0:
                    faces = app.get(person_crop)
                    if len(faces) != 0:
                        face = faces[0]
                        embedding = face.embedding
                        identity_results = vector_search(embedding, threshold=0.80)
                        name, rec_score = identity_results
                        rec_score = float(rec_score)
                        prediction_dict[track_id]["name"] = name
                        
                        # Red box for newly identified
                        cv2.rectangle(frame, (l, t), (r, b), (255, 0, 0), 2)
                        cv2.putText(frame, f"{name}-{rec_score:.2f}", (l, t - 10), 
                                  cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
                        
                        # Store detection
                        detection = {
                            "timestamp": round(timestamp, 3),
                            "frame_id": frame_id,
                            "camera_id": cam_id,
                            "track_id": track_id,
                            "person_id": name,
                            "bbox": [l, t, r, b],
                            "confidence": round(float(rec_score), 4),
                            "detection_confidence": round(float(score), 4) if score is not None else None
                        }
                        all_detections.append(detection)
            else:
                # Yellow box for already identified
                cv2.rectangle(frame, (l, t), (r, b), (0, 255, 255), 2)
                if score is not None:
                    cv2.putText(frame, f"{name}-{score:.2f}", (l, t - 10), 
                              cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2)
                else:
                    cv2.putText(frame, f"{name}", (l, t - 10), 
                              cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2)
                
                # Store detection
                detection = {
                    "timestamp": round(timestamp, 3),
                    "frame_id": frame_id,
                    "camera_id": cam_id,
                    "track_id": track_id,
                    "person_id": name,
                    "bbox": [l, t, r, b],
                    "confidence": None,
                    "detection_confidence": round(float(score), 4) if score is not None else None
                }
                all_detections.append(detection)
        
        # Write frame
        out.write(frame)
        frame_id += 1
        processed_frame_count += 1
        
        # GPU cleanup
        if ENABLE_GPU_CLEANUP and processed_frame_count % CLEANUP_INTERVAL == 0:
            if torch.cuda.is_available():
                torch.cuda.empty_cache()
            gc.collect()
        
        # Pause
        if PAUSE_EVERY_N_FRAMES > 0 and processed_frame_count % PAUSE_EVERY_N_FRAMES == 0:
            time.sleep(PAUSE_DURATION)
        
        # Progress update
        if processed_frame_count % 50 == 0:
            progress = (frame_id / total_frames * 100) if total_frames > 0 else 0
            print(f"  Progress: Frame {frame_id}/{total_frames} ({progress:.1f}%) | Detections: {len(all_detections)}")
    
    cap.release()
    out.release()
    
    # Final cleanup
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    gc.collect()
    
    print(f"\n✓ Completed {cam_id}:")
    print(f"   Total frames: {frame_id}")
    print(f"   Processed frames: {processed_frame_count}")
    print(f"   Detections: {len(all_detections)}")
    print(f"   Output video: {output_video_path}")
    
    return all_detections, output_video_path

print("✓ Processing function defined")


In [ ]:
# Process all videos
all_detections = []
camera_metadata = {}

for cam_id, config in sorted(CAMERA_CONFIG.items()):
    video_path = Path(VIDEO_DIR) / config["name"]
    
    if not video_path.exists():
        print(f"⚠ Warning: {video_path} not found, skipping {cam_id}")
        continue
    
    try:
        detections, output_path = process_video(cam_id, video_path, config)
        all_detections.extend(detections)
        
        # Store camera metadata
        camera_metadata[cam_id] = {
            "video_file": config["name"],
            "output_video": f"videos/{cam_id}.mp4",
            "location": config["location"]
        }
        
        # Save individual JSON
        individual_data = {
            "camera_id": cam_id,
            "metadata": {
                "total_detections": len(detections),
                "camera": camera_metadata[cam_id]
            },
            "detections": detections
        }
        
        individual_json_path = os.path.join(INDIVIDUAL_JSON_DIR, f"{cam_id}.json")
        with open(individual_json_path, 'w') as f:
            json.dump(individual_data, f, indent=2)
        
        print(f"✓ Saved individual results: {individual_json_path}")
        
    except Exception as e:
        print(f"❌ Error processing {cam_id}: {e}")
        import traceback
        traceback.print_exc()
        continue

print(f"\n{'='*60}")
print("PROCESSING COMPLETE")
print(f"{'='*60}")


## Step 7: Generate Final Results

Merge all detections into a single JSON file.


In [ ]:
# Sort all detections globally by timestamp
all_detections.sort(key=lambda x: (x["timestamp"], x["camera_id"], x["frame_id"]))

# Create final output structure
output_data = {
    "metadata": {
        "total_detections": len(all_detections),
        "total_cameras": len(camera_metadata),
        "cameras": camera_metadata
    },
    "detections": all_detections
}

# Save JSON
with open(OUTPUT_JSON, 'w') as f:
    json.dump(output_data, f, indent=2)

print(f"✓ Saved {len(all_detections)} detections from {len(camera_metadata)} cameras")
print(f"✓ Output saved to: {OUTPUT_JSON}")

# Print summary
print(f"\n{'='*60}")
print("SUMMARY")
print(f"{'='*60}")
print(f"Total detections: {len(all_detections)}")

# Count by person
person_counts = defaultdict(int)
for det in all_detections:
    person_counts[det["person_id"]] += 1

print(f"\nDetections by person:")
for person, count in sorted(person_counts.items()):
    print(f"  {person}: {count}")

# Count by camera
camera_counts = defaultdict(int)
for det in all_detections:
    camera_counts[det["camera_id"]] += 1

print(f"\nDetections by camera:")
for camera, count in sorted(camera_counts.items()):
    print(f"  {camera}: {count}")


## Step 8: Download Results

Download the processed videos and detection JSON files.


In [ ]:
# Create a zip file with all results
import zipfile
import shutil

zip_filename = "processed_results.zip"

print("📦 Creating results zip file...")
with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
    # Add detection JSON
    if os.path.exists(OUTPUT_JSON):
        zipf.write(OUTPUT_JSON, "detections.json")
        print(f"✓ Added detections.json")
    
    # Add individual JSON files
    individual_dir = Path(INDIVIDUAL_JSON_DIR)
    for json_file in individual_dir.glob("*.json"):
        zipf.write(json_file, f"individual/{json_file.name}")
        print(f"✓ Added {json_file.name}")
    
    # Add processed videos
    video_dir = Path(OUTPUT_VIDEO_DIR)
    for video_file in video_dir.glob("*.mp4"):
        zipf.write(video_file, f"videos/{video_file.name}")
        print(f"✓ Added {video_file.name}")

print(f"\n✅ Results zip created: {zip_filename}")
print(f"📥 File size: {os.path.getsize(zip_filename) / (1024*1024):.1f} MB")
print("\n⬇️  Downloading results...")
files.download(zip_filename)


In [ ]:
# Optionally download individual files
print("\n📥 Individual file downloads:")
print("   Run the cell below to download specific files")

# Uncomment to download individual files:
# files.download(OUTPUT_JSON)  # Download detections.json
# files.download("processed_data/videos/CAM_01.mp4")  # Download specific video
